In [16]:
import numpy as np
import pandas as pd
from ckiptagger import data_utils, WS, POS, NER, construct_dictionary
from ckip_transformers import __version__
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy
import pytextrank

In [13]:
#下載CKIP Tagger斷詞模型，下載完了已經不必再載第二次。
# data_utils.download_data_gdown(r"D:\Code\NTUT_Thesis")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: D:\Code\NTUT_Thesis\data.zip
100%|██████████| 1.88G/1.88G [07:36<00:00, 4.11MB/s]


In [ ]:
# Setting GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
#導入CKIP Transfomers
# Show version
print(__version__)

# Initialize drivers
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(model="albert-base", device=-1)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(model="albert-base", device=-1)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(model="albert-base", device=-1)
print("Initializing drivers ... all done")
print()

0.3.2
Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER
Initializing drivers ... all done



In [4]:
data = pd.read_csv('taiwan_charity_news_v2.csv', encoding='UTF-8', index_col='Index')

In [5]:
data.head(3)

,Date,Title,Media,Content,Temp_label
Index,,,,,
1,2017/1/9,視障弟月薪僅800？ 勞工局：是見習生獎金，不是薪資。,ETtoday_News,一名雙眼失明的闕小弟因為父親入獄、肢體障礙的母親也遠在高雄，只剩他和弟弟2人在北部生活，只好...,neutral
2,2017/1/13,開發金，邀九企業挺公益。,UDN_Database,開發金控多年來透過旗下中華開發工銀文教基金會及凱基社會慈善基金會，辦理各項公益活動。「年度公...,neutral
3,2017/3/3,月亮的孩子加強視力，把握黃金期。,UDN_Database,白化症患者體內缺乏黑色素，頭髮和皮膚雪白，因眼睛畏光、在夜間活動相對較舒適，印地安人稱他們為...,neutral


In [6]:
#隨意取一則新聞的content出來檢查
#Find Index==100 >>Keyword Extraction
test_string =data['Content'].loc[data.index == 100].values
print(test_string)

['「天下的媽媽都是一樣的」，誰說視障女性無法勝任媽媽的角色？\n愛盲基金會表示，基金會成立近30年，看見無數視障家庭面臨的困難與需求，也見證他們的努力，不少許多視障女性也面臨職場與家庭兩頭燒的諸多困境，愛盲服務的其中一位視障媽媽林佳臻，在視障、單親、低收入戶等窘迫條件下，15年來，不但一手撐起整個家、帶大4個孩子，甚至考取多項證照，實現自身志趣。\n愛盲基金會指出，在台灣領有視障手冊的近6萬人中，約87%的人屬於中途失明，林佳臻在15年前因生產過程併發子癇前症，造成視神經萎縮，雙眼視力值只剩0.3，視野中間有黑影擋住，當時身無分文，連辦理低收證明所需的工本費都湊不足。林佳臻說，當時全身上下連5塊錢都沒有，心想如果找到工作，一定要好好做。\n為了照顧一對龍鳳胎及另兩名幼女，林佳臻用米湯混奶粉以節省開銷，並爭取成為公園清潔代賑工，剛開始打掃時，常跌得滿身傷，總掃不乾淨，曾氣餒到想要摔掉掃把，但為了孩子咬牙忍下，一支掃把拿了12年，終於熬到龍鳳胎兒女今年升高一，放學後會主動到公園幫媽媽一起打掃。\n林佳臻說，眼盲並不可怕，心盲才是最可悲的，只要自己走出來，尋求可能的幫助，這也不可恥，她來到愛盲，學會使用白手杖，出門不再容易跌倒；還有可以放大500倍的放大鏡和擴視機，協助她讀書、考上證照。\n愛盲基金會副董事長黃克綸表示，愛盲今年集結了七位視障媽媽的故事，希望透過真實故事，讓大家看見任何一位必須兼顧職場與家庭的母親，都是不容易的，更何況是一位視障母親？很榮幸林佳臻站出來分享親身經歷，讓大家看見視障母親令人超乎想像的強大能量！也呼籲大家能與愛盲一起支持她們，成為所有視障家庭的強力後盾。\n愛盲基金會說，今年受疫情重創，募款陡降近3成，但視障服務的腳步卻不能一日稍停，呼籲社會各界善心人士用行動支持愛盲，一起成為全台灣逾6萬名視障者、逾百萬名低視能者的後援。']


In [7]:
#清理內文中的一些髒東西
for txts in data['Content']:
    data['Content'] = data['Content'].replace(' ', '').replace(',', '，').replace('(', '（').replace(')', '）').replace('……', '')
#
for txts in data['Title']:
    data['Title'] = data['Title'].replace(' ', '').replace(',', '，').replace('(', '（').replace(')', '）').replace('……', '')

In [8]:
#寫一個func去除某些詞性的詞彙
def wordfilter(sentence_ws, sentence_pos):
  short_with_pos = []
  short_sentence = []
  stop_pos = set(['Nep', 'Nh']) # 不保留的詞性包括指代定詞、 代名詞
  for word_ws, word_pos in zip(sentence_ws, sentence_pos):
    # 只留名詞和動詞
    is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
    # 去掉某些詞性
    is_not_stop_pos = word_pos not in stop_pos
    # 只剩一個字的詞也不留
    # is_not_one_charactor = not (len(word_ws) == 1)
    # 組成串列
    # if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
    if is_N_or_V and is_not_stop_pos:
      short_with_pos.append(f"{word_ws}({word_pos})")
      short_sentence.append(f"{word_ws}")
  return (" ".join(short_sentence), " ".join(short_with_pos))

In [27]:
#建立Tagger用專有詞典
word_to_weight = {
    "愛盲基金會": 2,
    "庇護工場": 1,
    "愛盲": 1,
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'愛盲': 1.0}), (4, {'庇護工場': 1.0}), (5, {'愛盲基金會': 2.0})]


In [28]:
#Tagger斷詞器測試
ws_test = WS("./data/", disable_cuda=True)

word_sentence_list = ws_test(
    test_string, 
    sentence_segmentation = True, 
    segment_delimiter_set = {"，", "。", "！", "？", "：", "；", "…", "、"}, 
    recommend_dictionary = dictionary, 
    )
print(word_sentence_list)

[['「', '天下', '的', '媽媽', '都', '是', '一樣', '的', '」', '，', '誰', '說', '視障', '女性', '無法', '勝任', '媽媽', '的', '角色', '？', '\n', '愛盲基金會', '表示', '，', '基金會', '成立', '近', '30', '年', '，', '看見', '無數', '視障', '家庭', '面臨', '的', '困難', '與', '需求', '，', '也', '見證', '他們', '的', '努力', '，', '不少', '許多', '視障', '女性', '也', '面臨', '職場', '與', '家庭', '兩', '頭', '燒', '的', '諸多', '困境', '，', '愛盲', '服務', '的', '其中', '一', '位', '視障', '媽媽', '林佳臻', '，', '在', '視障', '、', '單親', '、', '低收入戶', '等', '窘迫', '條件', '下', '，', '15', '年', '來', '，', '不但', '一手', '撐起', '整', '個', '家', '、', '帶', '大', '4', '個', '孩子', '，', '甚至', '考取', '多', '項', '證照', '，', '實現', '自身', '志趣', '。', '\n', '愛盲基金會', '指出', '，', '在', '台灣', '領有', '視障', '手冊', '的', '近', '6萬', '人', '中', '，', '約', '87%', '的', '人', '屬於', '中途', '失明', '，', '林佳臻', '在', '15', '年', '前', '因', '生產', '過程', '併發', '子癇前', '症', '，', '造成', '視神經', '萎縮', '，', '雙眼', '視力值', '只', '剩', '0.3', '，', '視野', '中間', '有', '黑影', '擋住', '，', '當時', '身', '無', '分文', '，', '連', '辦理', '低收', '證明', '所', '需', '的', '工本費', '都', '湊', '不足', '。', 

In [9]:
ws = ws_driver(test_string)
pos = pos_driver(ws)
ner = ner_driver(test_string)
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(test_string, ws, pos, ner):
    (short, res) = wordfilter(sentence_ws, sentence_pos)
    print("斷詞後：")
    print(short)
    print("斷詞後+詞性標注：")
    print(res)

Inference: 100%|██████████| 1/1 [00:02<00:00,  2.58s/it]

斷詞後：
天下 媽媽 一樣 說 視障 女性 勝任 媽媽 角色 愛盲 基金會 表示 基金會 成立 近 30 年 看見 無數 視障 家庭 面臨 困難 需求 見證 努力 不少 許多 視障 女性 面臨 職場 家庭 兩 頭 燒 諸多 困境 愛 盲 服務 一 位 視障 媽媽 林佳臻 視障 單親 低收入戶 窘迫 條件 下 15 年 來 撐起 整 個 家 帶大 4 個 孩子 考取 多 項 證照 實現 志趣 愛盲 基金會 指出 台灣 領有 視障 手冊 近 6萬 人 中 87% 人 屬於 中途 失明 林佳臻 15 年 前 生產 過程 併發 子 癇前症 造成 視神經 萎縮 雙眼 視力值 剩 0.3 視野 中間 有 黑影 擋住 當時 身 無 分文 辦理 低收 證明 需 工本費 湊 不足 林佳臻 說 當時 全身 上下 5 塊 錢 沒有 心想 找到 工作 做 照顧 一 對 龍鳳胎 另 兩 名 幼女 林佳臻 米湯 混 奶粉 節省 開銷 爭取 成為 公園 清潔 代賑工 開始 打掃 時 跌 滿 身 傷 掃 乾淨 氣餒 想要 摔掉 掃把 孩子 咬牙 忍下 一 支 掃把 拿 12 年 熬到 龍鳳胎 兒女 今年 升高 一 放學 後 主動 到 公園 媽媽 打掃 林佳臻 說 眼盲 可怕 心盲 可悲 走出來 尋求 可能 幫助 可恥 來到 愛 盲 學會 使用 白 手杖 出門 容 易 跌倒 有 放大 500 倍 放大鏡 擴視機 協助 讀書 考上 證照 愛盲 基金會 副董事長 黃克綸 表示 愛盲 今年 集結 七 位 視障 媽媽 故事 希望 真實 故事 讓 看見 任何 一 位 兼顧 職場 家庭 母親 容易 一 位 視障 母親 榮幸 林佳臻 站出來 分享 經歷 讓 看見 視障 母親 令 人 超乎 想像 強大 能量 呼籲 愛盲 支持 成為 所有 視障 家庭 後盾 愛盲 基金會 說 今年 疫情 重創 募款 陡降 3成 視障 服務 腳步 一 日 停 呼籲 社會 各界 善心 人士 行動 支持 愛盲 成為 全 台灣 逾 6萬 名 視障者 逾 百萬 名 低視能 者 後援
斷詞後+詞性標注：
天下(Nc) 媽媽(Na) 一樣(VH) 說(VE) 視障(VH) 女性(Na) 勝任(VJ) 媽媽(Na) 角色(Na) 愛盲(VL) 基金會(Nc) 表示(VE) 基金會(Nc) 成立(VC) 近(Nes) 30(Neu) 年(Nf) 看見(

In [10]:
# Keyword Extraction TF-IDF Method
# 建立 CountVectorizer 物件
vectorizer = CountVectorizer()

# 將 text 轉成詞頻矩陣
word_count = vectorizer.fit_transform([short])

# 建立 TfidfTransformer 物件
tfidf_transformer = TfidfTransformer()

# 將詞頻矩陣轉換成 TF-IDF 矩陣
tfidf_matrix = tfidf_transformer.fit_transform(word_count)

# 取得詞彙表
words = vectorizer.get_feature_names()

# 取得每個詞彙的 TF-IDF 值
tfidf_values = tfidf_matrix.toarray()[0]

# 取得排序後的索引位置
sorted_index = np.argsort(tfidf_values)[::-1]

# 取得排名前 10 的關鍵字
top_k = 10
top_k_words = [words[i] for i in sorted_index[:top_k]]

print(top_k_words)
print(tfidf_matrix.shape)

['視障', '愛盲', '林佳臻', '基金會', '媽媽', '家庭', '成為', '看見', '今年', '母親']
(1, 161)


In [15]:
# Keyword Extraction TextRank Method
# 要確認有先在環境執行python -m spacy download zh_core_web_sm 下載中文模型

nlp = spacy.load('zh_core_web_sm')

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

doc = nlp(short)

# extract the top 10 keywords
keywords = []
for p in doc._.phrases:
    if len(keywords) >= 10:
        break
    keywords.append(p.text)
    
print(keywords)

['3成', '今年', '台灣', '單親', '林佳臻', '讀書', '12 年', '15 年', '3', '4']


In [20]:
# Keyword Extraction LDA Method
# 設定LDA模型的參數
NUM_TOPICS = 1 #看要設定幾個主題
NUM_KEYWORDS = 10
MAX_ITER = 50

# 建立CountVectorizer物件，並使用它將文本轉換成詞頻矩陣
# vectorizer = CountVectorizer()
X = vectorizer.fit_transform([short])

# 建立LDA模型，並使用fit_transform方法將詞頻矩陣轉換成主題-詞頻分布矩陣
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=MAX_ITER)
lda.fit_transform(X)

# 取出每個主題的關鍵字
keywords = []
for topic_idx, topic in enumerate(lda.components_):
    top_keyword_idxs = topic.argsort()[:-NUM_KEYWORDS-1:-1]
    top_keywords = [vectorizer.get_feature_names()[idx] for idx in top_keyword_idxs]
    keywords.append(top_keywords)

# 顯示每個主題的關鍵字
for i, topic_keywords in enumerate(keywords):
    print(f"Topic #{i+1}: {' '.join(topic_keywords)}")

Topic #1: 視障 愛盲 林佳臻 基金會 媽媽 家庭 成為 看見 今年 母親
